In [60]:
import pandas as pd
# !pip install pyarrow

MESES = ['01','02','03','04','05','06','07','08','09','10','11','12']
ANOS = ['2022','2023']
COL_INTERESSE = ['Matricula','Nome','Orgão','Setor','Cargo','Provento','Date']


In [61]:
###Definindo Função para criar link dos aquivos desejados###
def url_mes_ano (mes, ano):
    mes=str(mes)
    ano=str(ano)
    api_url_prefix='https://api.portaltransparencia.sead.caucaia.ce.gov.br/funcionario/excel/1/'
    api_url_final=api_url_prefix + mes + '/' + ano
    return api_url_final


In [62]:
#### criando funcao pra coletar todos os dados ####
def download_all_data(meses:list, anos:list):
    df_geral = pd.DataFrame()
    for ano in anos:
        for mes in meses:
            try:
                df = pd.read_excel(url_mes_ano(mes,ano), header=1) # ler o arquivo diretamente do link gerado
                df['Date'] = pd.to_datetime(f'{ano}-{mes}-01') #cria coluna do mes e ano no formato ce data
                df_geral = pd.concat([df_geral,df])
                
            except:
                pass
    df = df_geral.reset_index()# retorna o DF com os indices corrigidos

    return df[COL_INTERESSE]

In [63]:
def converter_texto_em_float(texto:str):
    try:
        texto = texto.replace('R$','').strip()
        texto = texto.replace(',','.')
        return float(texto)
    except:
        return texto
    


In [64]:
def limpar_dados(df:pd.DataFrame):
    df = df.dropna(subset=['Nome']) #limpa as linhas que nao possuem nomes
    df['Provento'] = df['Provento'].map(converter_texto_em_float)#converter valores que possuem prefixo R$ em float
    #removendo no nomes cujos proventos sao igual a zero
    index_proventos_nulos = df[df['Provento']== 0].index.to_list() #coletando a lista de indices cujos os proventos sao zero
    for indice in index_proventos_nulos:
        df = df.drop(index=indice) #remove cada indice da lista
    df = df.reset_index() #reseta index para a ordem correta
    df = df[COL_INTERESSE]

    return df


In [65]:
#### carregando arquivos ###
file_name = 'dados.feather'

try:
    df = pd.read_feather(file_name)
except:
    print('Dados não encontrados. Realizando o download...')
    ####fazendo o download de tudo e salvando #####
    df = download_all_data(meses=MESES,anos=ANOS)
    df = limpar_dados(df)
    df.to_feather(file_name)
# df = limpar_dados(df=df)


Dados não encontrados. Realizando o download...


/tmp/ipykernel_9927/203533954.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Provento'] = df['Provento'].map(converter_texto_em_float)#converter valores que possuem prefixo R$ em float


In [67]:
#maximo valor pago a prefeitura
provento_max_global = df.describe().to_dict()['Provento']['max']
provento_mean_global = df.describe().to_dict()['Provento']['mean']

print(f'Provento maximo: R${provento_max_global}')
print(f'Provento medio: R${provento_mean_global:.2f}')
#mes com mais proventos
df_aux = df.groupby([pd.Grouper(key='Date',axis=0, freq='M')]).sum().sort_values('Provento', ascending=False).head(1)
provento_max_pago_mes_numero = df_aux['Provento'].keys().month.to_list()[0]
provento_max_pago_ano_numero = df_aux['Provento'].keys().year.to_list()[0]
provento_max_pago_mes_valor = df_aux['Provento'].to_list()[0]

print(f'O mairo valor pago foi no mes {provento_max_pago_mes_numero}/{provento_max_pago_ano_numero}, no valor de: R${provento_max_pago_mes_valor}')



Provento maximo: R$89166.66
Provento medio: R$4767.51
O mairo valor pago foi no mes 10/2022, no valor de: R$108521808.42


In [68]:
df_aux = df[['Setor', 'Provento']].groupby('Setor').sum().sort_values('Provento', ascending=False)
setor_maior_gasto = df_aux.head(1).index.to_list()[0]

In [69]:
#importando dash plotly
from dash import Dash, html, dash_table, dcc, Input, Output, callback
import plotly.express as px
import matplotlib.pyplot as plt
import dash_bootstrap_components as dbc
from dash.dash_table.Format import Format, Scheme, Sign

In [73]:
@callback(
    Output("graph", "figure"), Input("selection", "value")
)
def display_animated_graph(selection):
    animations = {
        "GDP - Scatter": px.scatter(
            df,
            x=df.index,
            y="Provento",
            animation_frame=df.index,
            animation_group='Setor',
            size="pop",
            color="continent",
            hover_name="country",
            log_x=True,
            size_max=55,
            range_x=[100, 100000],
            range_y=[25, 90],
        ),
        "Population - Bar": px.bar(
            df,
            x="continent",
            y="pop",
            color="continent",
            animation_frame="year",
            animation_group="country",
            range_y=[0, 4000000000],
        ),
    }
    return animations[selection]

In [72]:
STYLE01 = {'color':'white','text-align': 'center'}

#iniciar o app
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__,external_stylesheets=external_stylesheets)

#layout
app.layout = dbc.Container([
    dbc.Row([
        html.Br([])
        ]),
    dbc.Row([
        dbc.Col([
            html.H1('Acompanhamento de Funcionarios da Caucaia-CE',style=STYLE01)
        ],width={"size": 8, "offset": 2}),
        dbc.Col([
            html.Img(src='https://www.caucaia.ce.gov.br/imagens/logo.png', style={'width': '150px'}),
        ],width={"size": 1, "order": "last"}),

        html.Br([]),
        html.Br([])

        ]),
    dbc.Row([
        dbc.Col([html.H4('Maior Salário Pago:', 
                         style=STYLE01),
                html.H6(f'R$ {provento_max_global}', 
                         style=STYLE01)
                         ]),
        dbc.Col([html.H4('Salário Médio:', 
                         style=STYLE01),
                html.H6(f'R$ {provento_mean_global:.2f}', 
                         style=STYLE01)
                         ]),
        dbc.Col([html.H4('Maior Gasto Mensal:', 
                         style=STYLE01),
                html.H6(f'R$ {provento_max_pago_mes_valor} ({provento_max_pago_mes_numero}/{provento_max_pago_ano_numero})', 
                         style=STYLE01)
                         ]),
        dbc.Col([html.H4('Setor com Maior Gasto:', 
                         style=STYLE01),
                html.H6(f'{setor_maior_gasto}', style=STYLE01)
                         ]),
        ]),
    
    dbc.Row([
            html.Br([])
             ]),
    
    dbc.Row([
            html.Div([
        html.H4("Animated GDP and population over decades"),
        html.P("Select an animation:"),
        dcc.RadioItems(
            id="selection",
            options=["GDP - Scatter", "Population - Bar"],
            value="GDP - Scatter",
        ),
        dcc.Loading(dcc.Graph(id="graph"), type="cube"),
    ])


    ])

], style={'backgroundColor':'#c7254e'})

#run
if __name__=='__main__':
    app.run(debug=True)